## Matrix Factorization

### Import Libraries & Dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from tqdm.notebook import tqdm
import pandas as pd

!pip install opendatasets
import opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# username = 'alinourian'
# key = '4200c3f083710c14f67a2dab913f61e6'

opendatasets.download("https://www.kaggle.com/datasets/prajitdatta/movielens-100k-dataset/ml-100k/u.data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: alinourian
Your Kaggle Key: ··········


100%|██████████| 4.77M/4.77M [00:01<00:00, 3.96MB/s]


In [60]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('./movielens-100k-dataset/ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('./movielens-100k-dataset/ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')
ratings_base.drop( "unix_timestamp", inplace=True, axis=1) 
ratings_test.drop( "unix_timestamp", inplace=True, axis=1) 

users = pd.unique(ratings_base['user_id'])
movies = pd.unique(ratings_base['movie_id'])

print(ratings_base[0:20])

print('Number of traing rates:', ratings_base.shape[0])
print('Number of test rates:', ratings_test.shape[0])

    user_id  movie_id  rating
0         1         1       5
1         1         2       3
2         1         3       4
3         1         4       3
4         1         5       3
5         1         6       5
6         1         7       4
7         1         8       1
8         1         9       5
9         1        10       3
10        1        11       2
11        1        12       5
12        1        13       5
13        1        14       5
14        1        15       5
15        1        16       5
16        1        17       3
17        1        18       4
18        1        19       5
19        1        21       1
Number of traing rates: 90570
Number of test rates: 9430


### Create Ratings Matrix

In [61]:
train_df = ratings_base.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
test_df = ratings_test.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

user_sample = np.random.randint(len(users))
movie_sample = np.random.randint(len(movies))

N, M = train_df.shape
K = 5

train_df

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Classes & Funcitons

In [6]:
def matrix_factorization(R, P, Q, K, alpha=0.005, lambda_=0.0005, max_iter=50):
    '''
    R: rating matrix
    P: User features matrix
    Q: Item features matrix
    K: latent features
    max_iter: maximum number of iterations
    alpha: learning rate
    lambda_: regularization parameter
    '''
    grad_P = np.zeros_like(P)
    grad_Q = np.zeros_like(Q)
    E = R - np.dot(P, Q)

    for step in tqdm(range(max_iter)):
        for i in range(len(R)):
            for j in range(len(R[0])):
                if R[i][j] > 0:
                    for k in range(K):
                        grad_P[i][k] = grad_P[i][k] + alpha * (2 * E[i][j] * Q[k][j] - lambda_ * P[i][k])
                        grad_Q[k][j] = grad_Q[k][j] + alpha * (2 * E[i][j] * P[i][k] - lambda_ * Q[k][j])

        P += grad_P
        Q += grad_Q
        grad_P = np.zeros_like(P)
        grad_Q = np.zeros_like(Q)

        R_hat = np.dot(P,Q)
        E = R - R_hat
        E[R == 0] = 0

        e = np.sum(E ** 2) + (lambda_/2) * np.sum(P ** 2) + np.sum(Q ** 2)
        if (step+1) % 10 == 0:
            print(f'iteration: {step+1}, error: {e}')
        if e < 0.001:
            break

    return P, Q

In [120]:
def modified_matrix_factorization(R, P, Q, K, alpha_b=(0.0002, 0.0002), alphas=(0.0002, 0.0002),
                                  lambda_b=(0.01, 0.01), lambdas=(0.05, 0.05), max_iter=5000, predict=True):
    '''
    R: rating matrix
    P: User features matrix
    Q: Item features matrix
    K: latent features
    max_iter: maximum number of iterations
    alpha_b: tupe of bias of learning rate: (lr for P, lr for Q)
    alphas: tupe of learning rates: (lr for P, lr for Q)
    lambda_b: tupe of bias of regularization: (lr for P, lr for Q)
    lambda: tupe of regularizations: (lr for P, lr for Q)
    '''

    lambdas_P = lambdas[0] * np.ones((K,))
    lambdas_P[0] = 0
    lambdas_P[1] = lambda_b[0]

    lambdas_Q = lambdas[1] * np.ones((K,))
    lambdas_Q[0] = 0
    lambdas_Q[1] = lambda_b[1]

    alphas_P = alphas[0] * np.ones((K,))
    alphas_P[0] = 0
    alphas_P[1] = alpha_b[0]

    alphas_Q = alphas[1] * np.ones((K,))
    alphas_Q[0] = 0
    alphas_Q[1] = alpha_b[1]

    grad_P = np.zeros_like(P)
    grad_Q = np.zeros_like(Q)
    P[:, 0] = np.ones((P.shape[0],))
    Q[:, 1] = np.ones((Q.shape[0],))
    E = R - np.dot(P, Q)

    for step in tqdm(range(max_iter)):
        for i in range(len(R)):
            for j in range(len(R[0])):
                if R[i][j] > 0 or not predict:
                    for k in range(K):
                        grad_P[i][k] = grad_P[i][k] + alphas_P[k] * (2 * E[i][j] * Q[k][j] - lambdas_P[k] * P[i][k])
                        grad_Q[k][j] = grad_Q[k][j] + alphas_Q[k] * (2 * E[i][j] * P[i][k] - lambdas_Q[k] * Q[k][j])

        P += grad_P
        Q += grad_Q
        P[:, 0] = np.ones((P.shape[0],))
        Q[:, 1] = np.ones((Q.shape[0],))
        grad_P = np.zeros_like(P)
        grad_Q = np.zeros_like(Q)

        R_hat = np.dot(P,Q)
        E = R - R_hat
        E[R == 0] = 0

        e = np.sum(E ** 2) + (lambdas[0]/2) * np.sum(P ** 2) + (lambdas[1]/2) * np.sum(Q ** 2)
        if (step+1) % 10 == 0:
            print(f'iteration: {step+1}, error: {e}')
        if e < 0.001:
            break

    return P, Q

In [8]:
def modified_matrix_factorization_v2(R, P, Q, K, alpha_b=None, alphas=None, lambda_b=None, lambdas=None, max_iter=5000):
    '''
    R: rating matrix
    P: User features matrix
    Q: Item features matrix
    K: latent features
    max_iter: maximum number of iterations
    alpha_b: tupe of bias of learning rate: (lr for P, lr for Q)
    alphas: tupe of learning rates: (lr for P, lr for Q)
    lambda_b: tupe of bias of regularization: (lr for P, lr for Q)
    lambda: tupe of regularizations: (lr for P, lr for Q)
    '''

    n_users, n_movies = R.shape

    if alpha_b is None:
        alpha_b = f(n_users, n_movies, [0.0002 for _ in range(7)], K)
    else:
        alpha_b = f(n_users, n_movies, alpha_b, K)
    
    if alphas is None:
        alphas = f(n_users, n_movies, [0.0002 for _ in range(7)], K)
    else:
        alphas = f(n_users, n_movies, alphas, K)

    if lambda_b is None:
        lambda_b = f(n_users, n_movies, [0.0005 for _ in range(7)], K)
    else:
        lambda_b = f(n_users, n_movies, lambda_b, K)
    
    if lambdas is None:
        lambdas = f(n_users, n_movies, [0.0005 for _ in range(7)], K)
    else:
        lambdas = f(n_users, n_movies, lambdas, K)

    lambdas_P = lambdas[0] * np.ones((K,))
    lambdas_P[0] = 0
    lambdas_P[1] = lambda_b[0]

    lambdas_Q = lambdas[1] * np.ones((K,))
    lambdas_Q[0] = 0
    lambdas_Q[1] = lambda_b[1]

    alphas_P = alphas[0] * np.ones((K,))
    alphas_P[0] = 0
    alphas_P[1] = alpha_b[0]

    alphas_Q = alphas[1] * np.ones((K,))
    alphas_Q[0] = 0
    alphas_Q[1] = alpha_b[1]

    grad_P = np.zeros_like(P)
    grad_Q = np.zeros_like(Q)
    P[:, 0] = np.ones((P.shape[0],))
    Q[:, 1] = np.ones((Q.shape[0],))
    E = R - np.dot(P, Q)

    for step in tqdm(range(max_iter)):
        for i in range(len(R)):
            for j in range(len(R[0])):
                if R[i][j] > 0:
                    for k in range(K):
                        grad_P[i][k] = grad_P[i][k] + alphas_P[k] * (2 * E[i][j] * Q[k][j] - lambdas_P[k] * P[i][k])
                        grad_Q[k][j] = grad_Q[k][j] + alphas_Q[k] * (2 * E[i][j] * P[i][k] - lambdas_Q[k] * Q[k][j])

        P += grad_P
        Q += grad_Q
        P[:, 0] = np.ones((P.shape[0],))
        Q[:, 1] = np.ones((Q.shape[0],))
        grad_P = np.zeros_like(P)
        grad_Q = np.zeros_like(Q)

        R_hat = np.dot(P,Q)
        E = R - R_hat
        E[R == 0] = 0

        e = np.sum(E ** 2) + (lambdas[0]/2) * np.sum(P ** 2) + (lambdas[1]/2) * np.sum(Q ** 2)
        if (step+1) % 10 == 0:
            print(f'iteration: {step+1}, error: {e}')
        if e < 0.001:
            break

    return P, Q

In [112]:
class MatrixFactorization():
    def __init__(self, R, K, alpha, beta, iterations):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        training_process = []
        for i in tqdm(range(self.iterations)):
            np.random.shuffle(self.samples)
            self.sgd()
            sse = self.sse()
            training_process.append((i, sse))
            if (i+1) % 10 == 0:
                print(f'Iteration: {i+1} ; error={sse}')
        return training_process

    def sse(self):
        xs, ys = self.R.nonzero()
        predicted = self.matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2) + self.alpha / 2 * (np.sum(self.P ** 2) + np.sum(self.Q ** 2))
        return error

    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def matrix(self):
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [102]:
def get_valid_indexes(df):
    X = []
    for i in tqdm(df.columns):
        for j in df.index:
            if df[i].loc[j] != 0:
                X.append((j, i))
    return np.array(X)

test_samples = get_valid_indexes(test_df)

def predict_rate(train_df, test_df, predic_df, u, i):
    print(u, i)
    if train_df[i].loc[u] == 0:
        print(f'User {u} may rate movie {i} with score {R_hat[u-1][i-1]}', end='')
        try:
            true_label = test_df[i].loc[u]
            print(f'\t(true rate score: {true_label})\n')
        except:
            print(f'\t(no real label!)\n')
        
    else:
        print(f'User {u} has already rated movie {i} with scroe {train_df[i].loc[u]}', end='')
        print(f'\t(reconstructed score: {round(predic_df[i].loc[u], 2)})\n')

  0%|          | 0/1129 [00:00<?, ?it/s]

### Q3: GD

In [62]:
R = train_df.values

P = np.random.rand(N,K)
Q = np.random.rand(K,M)

P_n, Q_n = matrix_factorization(R, P, Q, K, alpha=0.0002, lambda_=0, max_iter=50)

R_hat = np.dot(P_n, Q_n)
R_hat_df = pd.DataFrame(R_hat, columns=train_df.columns, index=train_df.index)

  0%|          | 0/50 [00:00<?, ?it/s]

iteration: 10, error: 119676.2143000542
iteration: 20, error: 96462.25942155185
iteration: 30, error: 89418.25428683267
iteration: 40, error: 86100.0353645975
iteration: 50, error: 84182.6184390293


In [103]:
n = 5

r = np.random.randint(0, len(test_samples), n)
samples = test_samples[r]

for i in range(n):
    predict_rate(train_df, test_df, R_hat_df, samples[i][0], samples[i][1])

908 56
User 908 may rate movie 56 with score 3.6971350848920608	(true rate score: 4.0)

184 522
User 184 may rate movie 522 with score 3.5484324336294217	(true rate score: 3.0)

869 846
User 869 may rate movie 846 with score 2.2993223954497646	(true rate score: 2.0)

729 901
User 729 may rate movie 901 with score 1.0867675860385748	(true rate score: 1.0)

249 241
User 249 may rate movie 241 with score 4.003864579754146	(true rate score: 5.0)



### Q4

Suppose we are at the $(u, i)$-th training example, $r_{ui}$, and its approximation $\hat{r}_{ui}$ is given.
We compute the gradient of $e′
_{ui}$ for $k \in 1, ...,K$:

$$ \frac{\partial}{\partial p_{ik}} e′_{ij} = -2e_{ij} q_{kj} + \lambda p_{ik}$$

$$ \frac{\partial}{\partial q_{ik}} e′_{ij} = -2e_{ij} p_{ik} + \lambda q_{ik}$$


We update the weights in the direction opposite to the gradient:

$$ p'_{ik} = p_{ik} + \eta(2e_{ij} q_{kj} - \lambda p_{ik})$$

$$ q'_{ik} = q_{ik} + \eta(2e_{ij} p_{ik} - \lambda q_{ik})$$

that is, we change the weights in P and Q to decrease the square of actual error and keep
the weights of P and Q small. Note that opposed to Simon Funk, we train all values of P
and Q simultaneously.

### Q5: Add Regularization

In [104]:
R = train_df.values
 
P = np.random.rand(N,K)
Q = np.random.rand(K,M)

P_n, Q_n = matrix_factorization(R, P, Q, K, alpha=0.0002, lambda_=0.01, max_iter=50)

R_hat = np.dot(P_n, Q_n)
R_hat_df = pd.DataFrame(R_hat, columns=train_df.columns, index=train_df.index)

  0%|          | 0/50 [00:00<?, ?it/s]

iteration: 10, error: 118276.91356776915
iteration: 20, error: 95741.77622436266
iteration: 30, error: 88842.6945068997
iteration: 40, error: 85631.18745557584
iteration: 50, error: 83799.42193344288


In [110]:
n = 5

r = np.random.randint(0, len(test_samples), n)
samples = test_samples[r]

for i in range(n):
    predict_rate(train_df, test_df, R_hat_df, samples[i][0], samples[i][1])

302 309
User 302 may rate movie 309 with score 1.629425050936746	(true rate score: 2.0)

890 340
User 890 may rate movie 340 with score 3.914542076878311	(true rate score: 4.0)

555 1054
User 555 may rate movie 1054 with score 2.185736686411238	(true rate score: 3.0)

412 135
User 412 may rate movie 135 with score 3.9253094783207	(true rate score: 4.0)

825 544
User 825 may rate movie 544 with score 3.4836589682806145	(true rate score: 3.0)



### Q6

The formulas become the following:

$$ p'_{ik} = p_{ik} + \eta^p(u,i,k).(2e_{ij} q_{kj} - \lambda^p(u,i,k). p_{ik})$$

$$ q'_{ik} = q_{ik} + \eta^q(u,i,k).(2e_{ij} p_{ik} - \lambda^q(u,i,k). q_{ik})$$

### Q7: Modified Parameters

In [121]:
R = train_df.values

P = np.random.rand(N,K)
Q = np.random.rand(K,M)

P_n, Q_n = modified_matrix_factorization(R, P, Q, K, max_iter=50)

R_hat = np.dot(P_n, Q_n)
R_hat_df = pd.DataFrame(R_hat, columns=train_df.columns, index=train_df.index)

  0%|          | 0/50 [00:00<?, ?it/s]

iteration: 10, error: 117213.7824755528
iteration: 20, error: 93239.61335230025
iteration: 30, error: 85932.06278113519
iteration: 40, error: 82483.7826153798
iteration: 50, error: 80503.7077403546


In [126]:
n = 5

r = np.random.randint(0, len(test_samples), n)
samples = test_samples[r]

for i in range(n):
    predict_rate(train_df, test_df, R_hat_df, samples[i][0], samples[i][1])

921 313
User 921 may rate movie 313 with score 4.214057598480869	(true rate score: 5.0)

248 589
User 248 may rate movie 589 with score 3.6410754812962276	(true rate score: 4.0)

666 56
User 666 may rate movie 56 with score 3.949229574252867	(true rate score: 4.0)

266 25
User 266 may rate movie 25 with score 2.904109655614473	(true rate score: 3.0)

182 763
User 182 may rate movie 763 with score 3.366815924896129	(true rate score: 3.0)



### Q7-v2: Modified Parameters (Another Way)

In [144]:
R = train_df.values

N, M = R.shape
K = 5

mf = MatrixFactorization(R, K, 0.0002, 0.01, 50)
loss = mf.train()
R_hat_df = pd.DataFrame(mf.matrix(), columns=train_df.columns, index=train_df.index)

  0%|          | 0/50 [00:00<?, ?it/s]

Iteration: 10 ; error=102465.25268622927
Iteration: 20 ; error=95558.12467789053
Iteration: 30 ; error=91875.3448727701
Iteration: 40 ; error=89547.5677391141
Iteration: 50 ; error=87913.65005594605


In [145]:
n = 5

r = np.random.randint(0, len(test_samples), n)
samples = test_samples[r]

for i in range(n):
    predict_rate(train_df, test_df, R_hat_df, samples[i][0], samples[i][1])

358 896
User 358 may rate movie 896 with score 4.158991227710065	(true rate score: 4.0)

887 931
User 887 may rate movie 931 with score 2.5384082977228033	(true rate score: 3.0)

370 705
User 370 may rate movie 705 with score 3.6883384957078977	(true rate score: 3.0)

722 300
User 722 may rate movie 300 with score 3.9755323883679243	(true rate score: 3.0)

851 987
User 851 may rate movie 987 with score 2.1417579928401875	(true rate score: 1.0)



### Q8: Modified Parameters

In [129]:
def f(n_users, n_movies, ps, K):
    alpha = ps[0]
    alpha += ps[1] / np.log(n_users) + ps[2] / np.sqrt(n_users) + ps[3] / n_users
    alpha += ps[4] / np.log(n_movies) + ps[5] / np.sqrt(n_movies) + ps[6] / n_movies
    return alpha * np.ones((K,))

In [130]:
R = train_df.values

P = np.random.rand(N,K)
Q = np.random.rand(K,M)

P_n, Q_n = modified_matrix_factorization_v2(R, P, Q, K, max_iter=50)

R_hat = np.dot(P_n, Q_n)
R_hat_df = pd.DataFrame(R_hat, columns=train_df.columns, index=train_df.index)

  0%|          | 0/50 [00:00<?, ?it/s]

iteration: 10, error: 103923.01516817053
iteration: 20, error: 87351.21549088079
iteration: 30, error: 82228.32685481281
iteration: 40, error: 79778.46076675806
iteration: 50, error: 78332.9237378095


In [143]:
n = 5

r = np.random.randint(0, len(test_samples), n)
samples = test_samples[r]


for i in range(n):
    predict_rate(train_df, test_df, R_hat_df, samples[i][0], samples[i][1])

259 176
User 259 may rate movie 176 with score 4.151090989907249	(true rate score: 4.0)

621 107
User 621 may rate movie 107 with score 3.9000024262563984	(true rate score: 4.0)

336 405
User 336 may rate movie 405 with score 3.1952727799708134	(true rate score: 3.0)

810 313
User 810 may rate movie 313 with score 4.581602684964286	(true rate score: 5.0)

734 174
User 734 may rate movie 174 with score 3.992884982349202	(true rate score: 4.0)



### Check on some train samples

In [147]:
train_samples = get_valid_indexes(train_df)

  0%|          | 0/1680 [00:00<?, ?it/s]

In [152]:
n = 5

r = np.random.randint(0, len(train_samples), n)
samples = train_samples[r]

for i in range(n):
    predict_rate(train_df, test_df, R_hat_df, samples[i][0], samples[i][1])

521 174
User 521 has already rated movie 174 with scroe 4.0	(reconstructed score: 3.87)

514 186
User 514 has already rated movie 186 with scroe 4.0	(reconstructed score: 3.97)

479 479
User 479 has already rated movie 479 with scroe 4.0	(reconstructed score: 3.92)

234 151
User 234 has already rated movie 151 with scroe 3.0	(reconstructed score: 3.1)

374 576
User 374 has already rated movie 576 with scroe 3.0	(reconstructed score: 3.05)

